In [15]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split

import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [16]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
# Hyper-parameters
num_epochs = 10
batch_size = 1
learning_rate = 0.01

In [18]:
class IrisDataset(Dataset):

    # data loading
    def __init__(self):
        iris = datasets.load_iris()
        feature = pd.DataFrame(iris.data, columns=iris.feature_names)
        target = pd.DataFrame(iris.target, columns=['target'])
        iris_data = pd.concat([target, feature], axis=1)
        # Data type change and flatten targets
        self.x = torch.from_numpy(np.array(iris_data)[:, 1:].astype(np.float32))
        self.y = torch.from_numpy(np.array(iris_data)[:, [0]].astype(np.longlong).flatten())
        self.n_samples = self.x.shape[0]

    # working for indexing
    def __getitem__(self, index):
        
        return self.x[index], self.y[index]

    # return the length of our dataset
    def __len__(self):

        return self.n_samples


dataset = IrisDataset()

# create data spliter
def dataSplit(dataset, val_split=0.2, shuffle=False, random_seed=0):

    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(val_split * dataset_size))
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    
    train_indices, val_indices = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    return train_sampler , valid_sampler


train_sampler, valid_sampler = dataSplit(dataset = dataset, val_split = 0.2, shuffle = True, random_seed = 0)

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

In [19]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(in_features=4, out_features=24)
        self.fc2 = nn.Linear(in_features=24, out_features=24)
        self.fc3 = nn.Linear(in_features=24, out_features=3)

        
    def forward(self, net):
        net = F.relu(self.fc1(net))
        net = F.relu(self.fc2(net))
        net = self.fc3(net)

        return net

model = Network()
model.to(device)

Network(
  (fc1): Linear(in_features=4, out_features=24, bias=True)
  (fc2): Linear(in_features=24, out_features=24, bias=True)
  (fc3): Linear(in_features=24, out_features=3, bias=True)
)

In [20]:
lossfunc = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [21]:
min_valid_loss = np.inf
for epoch in range(num_epochs):
    train_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
      images = images.to(device)
      labels = labels.to(device)
      
      outputs = model(images)
      loss = lossfunc(outputs, labels)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      train_loss += loss.item()

           
    model.eval()
    n_correct = 0
    n_sample = 0
    with torch.no_grad():
      for (images, labels) in (val_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        val_loss = lossfunc(outputs, labels)

        predicted = model(images)
        predicted = torch.argmax(predicted,1)
        n_sample += labels.size(0)
        n_correct += ( labels.view(-1) == predicted ).sum().item()

    print('In epoch {}, loss: {:.3f}, val acc: {:.3f}, val_loss: {:.3f}'.format(
              epoch+1, train_loss/len(train_loader), n_correct/n_sample, val_loss))
    
    if min_valid_loss > val_loss:
      print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{val_loss:.6f}) \t Saving The Model')
      min_valid_loss = val_loss
      # Saving State Dict
      torch.save(model.state_dict(), 'saved_model.pth')

In epoch 1, loss: 1.081, val acc: 0.467, val_loss: 1.063
Validation Loss Decreased(inf--->1.062738) 	 Saving The Model
In epoch 2, loss: 0.968, val acc: 0.567, val_loss: 0.710
Validation Loss Decreased(1.062738--->0.709878) 	 Saving The Model
In epoch 3, loss: 0.693, val acc: 0.567, val_loss: 1.323
In epoch 4, loss: 0.502, val acc: 0.567, val_loss: 0.247
Validation Loss Decreased(0.709878--->0.246780) 	 Saving The Model
In epoch 5, loss: 0.419, val acc: 0.767, val_loss: 0.027
Validation Loss Decreased(0.246780--->0.027144) 	 Saving The Model
In epoch 6, loss: 0.358, val acc: 0.600, val_loss: 0.120
In epoch 7, loss: 0.298, val acc: 0.967, val_loss: 0.231
In epoch 8, loss: 0.315, val acc: 1.000, val_loss: 0.004
Validation Loss Decreased(0.027144--->0.004360) 	 Saving The Model
In epoch 9, loss: 0.293, val acc: 0.833, val_loss: 1.034
In epoch 10, loss: 0.248, val acc: 1.000, val_loss: 0.093


In [26]:
model = Network()
model.load_state_dict(torch.load('/content/saved_model.pth'))

correct = 0
with torch.no_grad():
    for data, target in val_loader:
        output = model(data)

        predicted = torch.argmax(output, 1)
        correct += (predicted == target).sum().item()
    
    accuracy = correct / len(val_loader)
    print(f'Test Accuracy: {accuracy}')     

Test Accuracy: 1.0
